In [17]:
!pip install matplotlib pillow

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


You should consider upgrading via the 'C:\Users\dyctw\.pyenv\pyenv-win\versions\3.10.5\python.exe -m pip install --upgrade pip' command.


In [1]:
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import os
from PIL import Image

# 設定波參數
time_steps = 50
amplitude = 0.5
wave_length = 2 * np.pi
k = 2 * np.pi / wave_length
omega = k

# 建立網格方塊
Nx, Ny, Nz = 40, 4, 4
x = np.linspace(0, 20, Nx)
y = np.linspace(-1, 1, Ny)
z = np.linspace(-1, 1, Nz)
X0, Y0, Z0 = np.meshgrid(x, y, z, indexing='ij')

def create_wireframe(ax, Xd, Yd, Zd, color):
    for j in range(Yd.shape[1]):
        for k_ in range(Zd.shape[2]):
            ax.plot(Xd[:, j, k_], Yd[:, j, k_], Zd[:, j, k_], color=color, linewidth=0.8)
    for i in range(Xd.shape[0]):
        for k_ in range(Zd.shape[2]):
            ax.plot(Xd[i, :, k_], Yd[i, :, k_], Zd[i, :, k_], color=color, linewidth=0.8)
    for i in range(Xd.shape[0]):
        for j in range(Yd.shape[1]):
            ax.plot(Xd[i, j, :], Yd[i, j, :], Zd[i, j, :], color=color, linewidth=0.8)

# 若無 frames 資料夾則建立
if not os.path.exists('frames'):
    os.makedirs('frames')

# 圖片大小設定
figsize = (5, 5)
dpi = 200

def generate_wave(pattern, title, displacement_func, color, output_folder):
    fig = plt.figure(figsize=figsize, dpi=dpi)
    fig.patch.set_facecolor('#333333')
    ax = fig.add_subplot(111, projection='3d')
    ax.set_facecolor('#333333')

    for frame in range(time_steps):
        ax.cla()
        ax.set_facecolor('#333333')
        ax.axis('off')
        ax.set_xlim(0, 20)
        ax.set_ylim(-1, 1)
        ax.set_zlim(-1, 1)
        ax.text2D(0.5, -0.05, title, fontsize=12, color='white', ha='center', transform=ax.transAxes)
        t = frame
        Xd, Yd, Zd = displacement_func(t)
        create_wireframe(ax, Xd, Yd, Zd, color)
        plt.savefig(f'{output_folder}/{pattern}_{frame:03d}.png', dpi=dpi, facecolor=fig.get_facecolor())
    plt.close(fig)

def p_wave_displacement(t):
    # P 波：粒子在 x 軸方向振動
    displacement = amplitude * np.sin(k * X0 - omega * t)
    return X0 + displacement, Y0, Z0

def s_wave_displacement(t):
    # S 波：粒子在 z 軸方向振動
    displacement = amplitude * np.sin(k * X0 - omega * t)
    return X0, Y0, Z0 + displacement

def love_wave_displacement(t):
    # 洛夫波：粒子在 y 軸方向振動
    Y_love = Y0 + amplitude * np.sin(k * X0 - omega * t)
    return X0, Y_love, Z0

def rayleigh_wave_displacement(t):
    # 雷利波：橢圓運動，x 方向與 z 方向同時振動
    X_rayleigh = X0 + amplitude * np.sin(k * X0 - omega * t)
    Z_rayleigh = Z0 + amplitude * np.cos(k * X0 - omega * t)
    return X_rayleigh, Y0, Z_rayleigh

# 生成波形
generate_wave('p_wave', ' ', p_wave_displacement, '#FFE153', 'frames')
generate_wave('s_wave', ' ', s_wave_displacement, '#79FF79', 'frames')
generate_wave('love_wave', ' ', love_wave_displacement, '#FF7575', 'frames')
generate_wave('rayleigh_wave', ' ', rayleigh_wave_displacement, '#97CBFF', 'frames')

def pngs_to_gif(pattern, output_gif, skip=0):
    frames_list = sorted([f for f in os.listdir('frames') if f.startswith(pattern)])
    frames_list = frames_list[skip:]  # 略過前幾幀

    images = []
    for f in frames_list:
        img = Image.open(os.path.join('frames', f)).convert("RGB")
        images.append(img)

    # 將 images 合成 GIF
    images[0].save(
        output_gif,
        save_all=True,
        append_images=images[1:],
        loop=0,
        duration=100
    )

# 合成 GIF
pngs_to_gif('p_wave', 'p_wave.gif', skip=20)
pngs_to_gif('s_wave', 's_wave.gif', skip=20)
pngs_to_gif('love_wave', 'love_wave.gif', skip=20)
pngs_to_gif('rayleigh_wave', 'rayleigh_wave.gif', skip=20)

print("已完成波形 GIF 生成。")


已完成波形 GIF 生成。
